# Narrowcasting

This notebook contains the experiment to show a **narrowcasting example**.

The notebook is organized as follows:

* In Section 1, we define all the simulation parameters.
* In Section 2, we run all the simulations needed for the experiment.
* In Section 3, we plot the results.

To just regenerate the figures from a summary file containing all the simulation objects of the experiment, you can skip Section 2, and only run the cells in Sections 1 & 3. 

---

## 1. Define the experiment parameters

#### Import libs

In [ ]:
%load_ext autoreload
%autoreload 2
import sys
if '..' not in sys.path:
    sys.path.append('..')

In [ ]:
import numpy as np
import pandas as pd
import pickle
import multiprocessing

In [ ]:
from lib.measures import *
from lib.experiment import run_experiment, save_summary, load_summary
from lib.calibrationSettings import calibration_lockdown_dates
from lib.calibrationFunctions import get_calibrated_params

## 1. General settings  

#### Set the random seed for reproducibility

In [ ]:
# Choose random seed
c = 0
np.random.seed(c)
TO_HOURS = 24.0

# Define prefix string used to save plots
expstr = 'narrowcasting'
multi_beta_calibration = False

#### Set the number of roll-outs to simulate

In [ ]:
random_repeats = 1 # For narrowcasting, only need 1 random restart, as only 1 realization is visualized

#### Set the time to simulate

In [ ]:
start_date = '2020-03-08'
end_date = '2020-03-27'

#### Define locations, mobility settings used for simulation, and experiment parameters

In [ ]:
locs = {
    'GER': {
        'TU': 'lib/mobility/Tubingen_settings_10.pk',
        'KL': 'lib/mobility/Kaiserslautern_settings_10.pk',
        'RH': 'lib/mobility/Ruedesheim_settings_10.pk', 
        'TR': 'lib/mobility/Tirschenreuth_settings_10.pk', 
    },
    'CH': {
        'VD': 'lib/mobility/Lausanne_settings_10.pk', 
        'LU': 'lib/mobility/Lucerne_settings_5.pk', 
        'TI': 'lib/mobility/Locarno_settings_2.pk', 
        'JU': 'lib/mobility/Jura_settings_10.pk',
    }
}

seed_summary_path = None

# experiment parameters

# TBD
# depending on the experiment

## 2. Run the simulations

Use settings as above and simulate in the future with additional measures

**WARNING: the following cells might take a long time to run depending of the parameters defined above!**

In [ ]:
for country, areas in locs.items():
    for area, mob_settings in areas.items():
                
        # load social distancing parameter
        p_stay_home = get_calibrated_params(country=country, area=area, multi_beta_calibration=multi_beta_calibration)['p_stay_home']
        
        # measures
        m = [
            # TBD; depending on the experiment
        ] 
        
        # simulate
        res, mob = run_experiment(
            country=country, 
            area=area, 
            mob_settings=mob_settings,
            start_date=start_date, 
            end_date=end_date, 
            measure_list=m, 
            random_repeats=random_repeats,
            test_update=None, 
            seed_summary_path=seed_summary_path,
            return_mob=True,
            multi_beta_calibration=multi_beta_calibration)
                
        # save results
        save_summary((res, mob), f'state-{expstr}--{country}-{area}.pk')
        
        print(country, area, ' done.', flush=True)
        
        break
        
    break

## 3. Plot the results

Import libs

In [ ]:
from lib.town_maps import MapIllustrator

Specify which area and time of simulation should be considered, and import correct bounding box.

In [ ]:
day_start = 7
day_duration = 14
country = 'GER'
area = 'TU'

In [ ]:
from lib.settings.town_settings_tubingen import bbox

Create map

In [ ]:
# add mobility traces to summary file because needed for plotting 
summary, mob = load_summary(f'state-{expstr}--{country}-{area}.pk')
summary.mob = [mob]

# plot
ill = MapIllustrator()
ill.marker_radius = 10.0
map_name = f'{expstr}--{country}-{area}'

sitemap = ill.empirical_infection_probability_map(
    bbox=bbox, 
    site_loc=mob.site_loc, 
    site_type=mob.site_type, 
    site_dict=mob.site_dict, 
    map_name=map_name, 
    sim=summary, 
    t0=TO_HOURS * day_start, 
    t1=TO_HOURS * (day_start + day_duration), 
    delta=0.0, 
    r=0)

In [ ]:
sitemap